In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [ ]:
# Generate summary statistics
df_market_data.describe()

In [3]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [4]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])

In [5]:
df_market_data.columns

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')

In [6]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']
)

# Copy the crypto names from the original DataFrame
df_market_data_scaled["coin_id"] = df_market_data.index


# Set the coin_id column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display the scaled DataFrame
display(df_market_data_scaled.head())

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [7]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))
print(k_values)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [8]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, n_init=10, random_state=42)
    
    # 2. Fit the model to the data using `df_market_data_scaled`
    kmeans_model.fit(df_market_data_scaled)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans_model.inertia_)
    
    print(f"Completed k={k:2d} | Inertia: {kmeans_model.inertia_:} | "
          f"Centers shape: {kmeans_model.cluster_centers_.shape}")


Completed k= 1 | Inertia: 287.0 | Centers shape: (1, 7)
Completed k= 2 | Inertia: 195.82021818036043 | Centers shape: (2, 7)
Completed k= 3 | Inertia: 123.19048183836958 | Centers shape: (3, 7)
Completed k= 4 | Inertia: 79.02243535120977 | Centers shape: (4, 7)
Completed k= 5 | Inertia: 65.302379141625 | Centers shape: (5, 7)
Completed k= 6 | Inertia: 54.7399219421002 | Centers shape: (6, 7)
Completed k= 7 | Inertia: 43.48160605008147 | Centers shape: (7, 7)
Completed k= 8 | Inertia: 37.517032490103574 | Centers shape: (8, 7)
Completed k= 9 | Inertia: 32.591890976724585 | Centers shape: (9, 7)
Completed k=10 | Inertia: 28.22289929060893 | Centers shape: (10, 7)
Completed k=11 | Inertia: 24.116198421409198 | Centers shape: (11, 7)


In [9]:
# Create a dictionary with the data to plot the Elbow curve
market_data = {
    "k": k_values,
    "inertia": inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
new_market_df = pd.DataFrame(market_data)

In [10]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
market_data_plot = new_market_df.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k_values)
market_data_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [11]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, n_init=10, random_state=42)

In [12]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, n_init=10, random_state=42)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
market_clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(market_clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [14]:
# Create a copy of the scaled DataFrame
df_market_data_scaled_copy = df_market_data_scaled.copy()

In [15]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_scaled_copy["market_clusters"] = market_clusters

# Display the copy of the scaled DataFrame
df_market_data_scaled_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,market_clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,2
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,2
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [16]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

scatter_plot = df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="market_clusters",  # Color points by cluster assignment
    hover_cols=["coin_id"],  # Show crypto name on hover
    width=800,
    height=500,
    title="Cryptocurrency Price Changes: 24h vs 7d (Clustered)",
    xlabel="24h Price Change (%)",
    ylabel="7d Price Change (%)",
    legend='top_right',
    colorbar=True,
    size=70,
    alpha=0.7
)


scatter_plot

:NdOverlay   [market_clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [18]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
market_data_pca[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [19]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
print("Explained variance ratio:")
print(pca.explained_variance_ratio_)
print(f"\nTotal explained variance: {sum(pca.explained_variance_ratio_):.2%}")

Explained variance ratio:
[0.3719856  0.34700813 0.17603793]

Total explained variance: 89.50%


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [39]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(
    data= market_data_pca,  # The PCA-transformed data from previous step
    columns=['PC1', 'PC2', 'PC3']
)

# Copy the crypto names from the original scaled DataFrame
df_market_data_pca['coin_id'] = df_market_data_scaled.index

# Set the coin_id column as index
df_market_data_pca.set_index('coin_id', inplace=True)

# Display the scaled PCA DataFrame
display(df_market_data_pca.head())

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [40]:
# Create a list with the number of k-values from 1 to 11
new_k_values= list(range(1, 12))
print(new_k_values)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [41]:
# Create an empty list to store the inertia values

new_inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in new_k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, n_init=10, random_state=42)
    
    # 2. Fit the model to the data using `df_market_data_pca`
    kmeans_model.fit(df_market_data_pca)
    
    # 3. Append the model.inertia_ to the inertia list
    new_inertia_values.append(kmeans_model.inertia_)
    
    print(f"Completed k={k:2d} | Inertia: {kmeans_model.inertia_:} | "
          f"Centers shape: {kmeans_model.cluster_centers_.shape}")

Completed k= 1 | Inertia: 256.8740855678923 | Centers shape: (1, 3)
Completed k= 2 | Inertia: 165.90199402036 | Centers shape: (2, 3)
Completed k= 3 | Inertia: 93.77462568057292 | Centers shape: (3, 3)
Completed k= 4 | Inertia: 49.66549665179733 | Centers shape: (4, 3)
Completed k= 5 | Inertia: 37.8394659868124 | Centers shape: (5, 3)
Completed k= 6 | Inertia: 30.777746141441167 | Centers shape: (6, 3)
Completed k= 7 | Inertia: 21.13405603747359 | Centers shape: (7, 3)
Completed k= 8 | Inertia: 17.091636643864735 | Centers shape: (8, 3)
Completed k= 9 | Inertia: 13.681139692992737 | Centers shape: (9, 3)
Completed k=10 | Inertia: 10.630647573870952 | Centers shape: (10, 3)
Completed k=11 | Inertia: 8.12862032021349 | Centers shape: (11, 3)


In [42]:
# Create a dictionary with the data to plot the Elbow curve
crypto_data_pca = {
    "k": k_values,
    "inertia": inertia_values
}
# Create a DataFrame with the data to plot the Elbow curve
df_crypto_pca = pd.DataFrame(crypto_data_pca)

In [43]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
crypto_plot_pca = df_crypto_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using Scaled PCA Data", xticks=new_k_values)
crypto_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**  No

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [50]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, n_init=10, random_state=42)

In [51]:
# Fit the K-Means model using the PCA data
k_model.fit(df_market_data_pca)

KMeans(n_clusters=4, n_init=10, random_state=42)

In [52]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
crypto_clusters = k_model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(crypto_clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [53]:
# Create a copy of the scaled PCA DataFrame
df_pca_copy = df_market_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_pca_copy["crypto_clusters"] = crypto_clusters

# Display the copy of the scaled PCA DataFrame
df_pca_copy.head()

,PC1,PC2,PC3,crypto_clusters
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,2
ripple,-0.471835,-0.222660,-0.479053,2
bitcoin-cash,-1.157800,2.041209,1.859715,0


In [54]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

scatter_plot = df_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="crypto_clusters",  # Color points by cluster assignment
    hover_cols=["coin_id"],  # Show crypto name on hover
    width=800,
    height=500,
    title="Cryptocurrency PCA Data",
    xlabel="Principal Component 1",
    ylabel="Principal Component 2",
    legend='top_right',
    colorbar=True,
    size=70,
    alpha=0.7
)



scatter_plot

:NdOverlay   [crypto_clusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 